In [1]:
import logging
import datetime

import numpy as np
import torch
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import matplotlib.pyplot as plt

import config
import modules.dataloaders as dataloaders

# import modules.brevitas.model_mobilenetv2_original_Brevitas as cnv_model
# import modules.brevitas.model_mobilenetv2_1M_Brevitas as cnv_model

# import modules.brevitas.model_mobilenetv2_mini_Brevitas as cnv_model
# import modules.brevitas.model_mobilenetv2_mini_160_Brevitas as cnv_model
# import modules.brevitas.model_mobilenetv2_mini_112_Brevitas as cnv_model

# import modules.brevitas.model_mobilenetv2_mini_Resnet_Brevitas as cnv_model
# import modules.brevitas.model_mobilenetv2_mini_Resnet_160_Brevitas as cnv_model
import modules.brevitas.model_mobilenetv2_mini_Resnet_112_Brevitas as cnv_model
# import modules.brevitas.model_mobilenetv2_mini_Resnet_ReLU6_Brevitas as cnv_model 
# import modules.brevitas.model_mobilenetv2_mini_Resnet_Sparse24_Brevitas as cnv_model
# import modules.brevitas.model_mobilenetv2_mini_Resnet_4bitINP_Brevitas as cnv_model

# import modules.brevitas.model_mobilenetv2_mini_Resnet_reluPerChannel_Brevitas as cnv_model


import modules.loss as loss
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch
import modules.utils as utils

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [2]:
# from brevitas.export import export_onnx_qcdq
# from brevitas.export import export_qonnx

# Logger

In [3]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info(f'{config.MODEL} Classifier.\n' +  
            '\tOne Head.\n' +
            '\tWeighted for Precision.\n' +
            '\tBrevitas Default.\n'+ 
            '\tDataset images divided by 255.\n')

# Hyperparameters Log

In [4]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('\nIMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nBrevitas Config:')
print(f'\tFixed Point: {config.FIXED_POINT}')
print(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
print(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
print(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
print(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('\nIMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nBrevitas Config:')
logger.info(f'\tFixed Point: {config.FIXED_POINT}')
logger.info(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
logger.info(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')


Datasets Length
	Train and Val: 128

Load Model: False
Device: cuda
Optimizer:
	Learning Rate: 0.001
	Weight Decay: 0.0
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 2
	Scheduler threshold: 0.001
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 3

IMG DIMS:
	Width: 112
	Height: 112

Brevitas Config:
	Fixed Point: True
	Weights Bit Width: 4
	Big Layers Weights Bit Width: 4
	Bias Bit Width: 4
	Activations Bit Width: 4


# Dataloaders

In [5]:
train_loader = dataloaders.get_train_loader()
val_loader = dataloaders.get_val_loader()


TRAIN DFIRE dataset
DFire Removed wrong images: 0
DFire empty images: 56
DFire only smoke images: 37
DFire only fire images: 8
DFire smoke and fire images: 27

Train DFire dataset len: 128

TRAIN FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 58
FASDD only smoke images: 22
FASDD only fire images: 2
FASDD smoke and fire images: 46

Train FASDD UAV dataset len: 128

VAL FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 59
FASDD only smoke images: 26
FASDD only fire images: 2
FASDD smoke and fire images: 41

Val FASDD UAV dataset len: 128

TRAIN FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 53
FASDD only smoke images: 36
FASDD only fire images: 19
FASDD smoke and fire images: 20

Train FASDD CV dataset len: 128

Val FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 50
FASDD only smoke images: 32
FASDD only fire images: 20
FASDD smoke and fire images: 26

Val FASDD CV dataset len: 128

==================== CONCAT

### Datasets Length

In [6]:
logger.info("\n********* Datasets Length *********")

print(f'Train Dataset Length: {len(train_loader.dataset)}')
logger.info(f'Train Dataset Length: {len(train_loader.dataset)}')

print(f'Test Dataset Length: {len(val_loader.dataset)}')
logger.info(f'Test Dataset Length: {len(val_loader.dataset)}')

Train Dataset Length: 640
Test Dataset Length: 384


### Plot Some Train Pictures

In [7]:
for i, (img, label) in enumerate(train_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
    #plt.show()
    plt.close()
    break

### Plot Some Val Pictures

In [8]:
for i, (img, label) in enumerate(val_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
    plt.close()
    break

# Load Model

In [9]:
# import importlib
# importlib.reload(cnv_model)

In [10]:
# model = cnv_model.MobileNetV2_MINI().to(config.DEVICE)
# model = cnv_model.MobileNetV2_MINI_160().to(config.DEVICE)
# model = cnv_model.MobileNetV2_MINI_112().to(config.DEVICE)

# model = cnv_model.MobileNetV2_1M().to(config.DEVICE)

# model = cnv_model.MobileNetV2_MINI_RESNET().to(config.DEVICE)
# model = cnv_model.MobileNetV2_MINI_RESNET_160().to(config.DEVICE)
model = cnv_model.MobileNetV2_MINI_RESNET_112().to(config.DEVICE)

# model = cnv_model.MobileNetV2_MINI_RESNET_RELU6().to(config.DEVICE)
# model = cnv_model.MobileNetV2_MINI_RESNET_SPARSE24().to(config.DEVICE)
# model = cnv_model.MobileNetV2_MINI_RESNET_4bitINP().to(config.DEVICE)

#### This model with WMUL 0.1 was changed to module model_mobilenetv2_original_wmul_01_Brevitas
# model = cnv_model.MobileNetV2_ORI_RESNET(
#     width_mult = 0.1
# ).to(config.DEVICE)
# model = cnv_model.MobileNetV2_ORI_RESNET(width_mult = config.WIDTH_MULT).to(config.DEVICE)

# model = cnv_model.MobileNetV2_MINI_RESNET_ACT_PerChannel().to(config.DEVICE)

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/brevitas/nn/mixin/base.py:77: UserWarning: Keyword arguments are being passed but they not being used.
  warn('Keyword arguments are being passed but they not being used.')


### Optimizer and Scheduler

In [11]:
optimizer = optim.Adam(model.parameters(), 
                       lr=config.LEARNING_RATE, 
                       weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)



### Parameters

In [12]:
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 68914
Total parameters = 68914



### Check Model Shape: Random Input

In [13]:
dummy_input = np.random.rand(4, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)
dummy_input = torch.tensor(dummy_input, dtype=torch.float32, device=config.DEVICE)
out_test = model(dummy_input)
print(f'Model shape is {out_test}')
#print(f'BED Model Arquitecture\n{cnv_model}')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


Model shape is tensor([[ 5.6331, -1.8161],
        [ 6.0725, -3.0223],
        [ 6.2509, -1.5729],
        [ 5.3889, -0.9621]], device='cuda:0', grad_fn=<AddmmBackward0>)


### Torchinfo

In [14]:
print(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))
logger.info(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                                                           Output Shape              Param #
MobileNetV2_MINI_RESNET_112                                                      [1, 2]                    --
├─Sequential: 1-1                                                                [1, 64, 7, 7]             60,206
│    └─QuantIdentity: 2-1                                                        [1, 3, 112, 112]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                       [1, 3, 112, 112]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                       [1, 3, 112, 112]          1
├─Sequential: 1-72                                                               --                        (recursive)
│    └─QuantReLU: 2-88                                                           --                        (recursive)
│    │    └─ActQuantProxyFromInjector: 3-96                                      --           

## Load Checkpoint

In [15]:
if config.LOAD_MODEL:
    print(f'Loading Checkpoint: \n{config.LOAD_MODEL_FILE}')
    logger.info(f'Loading Checkpoint: \n{config.LOAD_MODEL_FILE}')
    model.eval();
    epochs_trained = utils.load_checkpoint(
        config.LOAD_MODEL_FILE, 
        model=model, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        device=config.DEVICE)
    print(f'Loading model trained {epochs_trained} epochs')
    logger.info(f'Loading model trained {epochs_trained} epochs')

In [16]:
model.to(config.DEVICE);

# Loss Function

In [17]:
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    logger.info(f'\nLoss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    logger.info(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Loggers and Plotters for Losses and Metrics

In [18]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Main Function to Train

In [19]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_smoke_precision = 0. #torch.tensor([0.])
    smoke_f1_min_save = 0.9 #torch.tensor([0.9])
    best_mean_f1 = 0.
    best_mean_f1_epoch = 0

    if start_epoch == 0:
        epochs_plot = []
    else:
        epochs_plot = [i for i in range(start_epoch)]    

    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
                
        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            device=config.DEVICE)
        
        train_losses_logger.update_metrics(train_losses)
        train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN Stats", train_losses, train_metrics))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                device=config.DEVICE)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_metrics(val_losses)
            val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL Stats", val_losses, val_metrics))
            
        epochs_plot.append(epoch)

        loss_plotter.plot_all_metrics(
            train_losses_logger.get_metrics(),
            val_losses_logger.get_metrics(),
            epochs_plot)

        metrics_plotter.plot_all_metrics(
            train_metrics_logger.get_metrics(),
            val_metrics_logger.get_metrics(),
            epochs_plot)

        lr_logger.plot_lr(epochs_plot)
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.4f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.4f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  #+ '.onnx'
            utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # # Save model if precision increases and F1 > 0.9
        # if ( best_smoke_precision < val_metrics['Precision'][0] ) and ( val_metrics['F1'][0] > smoke_f1_min_save ) :
        #     best_smoke_precision = val_metrics['Precision'][0]
        #     print(f"\nSaving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     logger.info(f"Saving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     save_precision_name = f'best_smoke__precision={np.round(best_smoke_precision, decimals=4)}__epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_precision_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
        #     save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_precision_name #+ '.onnx'
        #     utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
            
        # Save model if precision > 0.9 and recall > 0.9
        # if ( val_metrics['Precision'][0] > 0.9 ) and ( val_metrics['Recall'][0] > 0.9 ) :
        #     print("\nSaving model with precision > 0.9 and recall > 0.9")
        #     logger.info("Saving model with precision > 0.9 and recall > 0.9")
        #     save_pre_name = f'smoke__precision={np.round(val_metrics["Precision"][0], decimals=4)}__' 
        #     save_rec_name = f'recall={np.round(val_metrics["Recall"][0], decimals=4)}__'
        #     save_pre_rec_name = save_pre_name + save_rec_name + f'epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
        #     save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name #+ '.onnx'
        #     utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # Save model if best mean F1 increases
        val_f1_mean = (val_metrics['F1'][0] + val_metrics['F1'][1]) / 2
        if (val_f1_mean > best_mean_f1) :
            best_mean_f1 = val_f1_mean
            best_mean_f1_epoch = epoch
            print(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            logger.info(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            save_f1_name = 'best_mean_F1'
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name #+ '.onnx'
            utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_classifier.pt') 

    # Best F1 Mean epoch
    print(f'\nBest F1 Mean at epoch {best_mean_f1_epoch}: {best_mean_f1}')
    logger.info(f'\n_______________________ Best F1 Mean at epoch {best_mean_f1_epoch}: {best_mean_f1} _______________________')

    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

In [20]:
# print(len(val_losses_logger.total))

# Training Loop

In [21]:
if __name__ == "__main__":
    
    print("Starting script\n")
    logger.info("Starting script\n")
    
    model = train_loop(model)

Starting script


***Start Training: 22:37:54


=== EPOCH 0/2 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.76it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
142.983     |100.212     |42.771      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.29it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
93.966      |40.432      |53.534      
SMOKE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.5104 - F1: 0.0000
FIRE -> Precision: 0.3021 - Recall: 1.0000 - Accuracy: 0.3021 - F1: 0.4640

Saving model with new best validation loss: 93.9661
Model exported to ONNX: experiments_brevitas/test_v22_mini_resnet_70k_112_128_ds/onnx/Mobilenetv2_Mini_Resnet_4bitINP_classifier__best_loss
Saving model with best Mean F1: 0.2320
Model exported to ONNX: experiments_brevitas/test_v22_mini_resnet_70k_112_128_ds/onnx/Mobilenetv2_Mini_Resnet_4bitINP_classifier__best_mean_F1

=== EPOCH 1/2 ===
Learning Rate = 0.001



...

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
104.484     |42.076      |62.408      
SMOKE -> Precision: 0.4948 - Recall: 0.7606 - Accuracy: 0.5026 - F1: 0.5996
FIRE -> Precision: 0.3021 - Recall: 1.0000 - Accuracy: 0.3021 - F1: 0.4640
Saving model with best Mean F1: 0.5318
Model exported to ONNX: experiments_brevitas/test_v22_mini_resnet_70k_112_128_ds/onnx/Mobilenetv2_Mini_Resnet_4bitINP_classifier__best_mean_F1

=== EPOCH 2/2 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.81it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
76.804      |42.815      |33.989      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.25it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
87.141      |39.399      |47.743      
SMOKE -> Precision: 0.4789 - Recall: 0.1809 - Accuracy: 0.5026 - F1: 0.2625
FIRE -> Precision: 0.3861 - Recall: 0.8621 - Accuracy: 0.5443 - F1: 0.5333

Saving model with new best validation loss: 87.1413
Model exported to ONNX: experiments_brevitas/test_v22_mini_resnet_70k_112_128_ds/onnx/Mobilenetv2_Mini_Resnet_4bitINP_classifier__best_loss

Best F1 Mean at epoch 1: 0.5317903459072113

***Script finished: 22:38:16

Time elapsed: 0:00:21.725467


In [22]:
# if __name__ == "__main__":
    
#     print("Train More script\n")
#     logger.info("Train More\n")
    
#     model = train_loop(model, start_epoch=75, epochs_to_train=15)

# Test with DFire MINI Dataset: Train and Test

In [23]:
# import importlib
# importlib.reload(config)
# importlib.reload(dataloaders)

In [24]:
train_dfire_mini_loader = dataloaders.get_dfire_mini_train_loader()
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TRAIN DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 20
DFire only smoke images: 45
DFire only fire images: 5
DFire smoke and fire images: 30

Test dataset len: 100

TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Load Checkpoint with Best F1 Mean

In [25]:
model_path = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__best_mean_F1.pt'

In [26]:
epochs_trained = utils.load_checkpoint(
    model_path, 
    model=model, 
    optimizer=optimizer, 
    scheduler=scheduler, 
    device=config.DEVICE)

Loading Model. Trained during 1 epochs


In [27]:
logger.info(f'\nLoading best F1 Mean model, trained for {epochs_trained} epochs')  

In [28]:
model.to('cuda');

### Whole Test Loader, to check it is the same as training

In [29]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.18it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
104.484     |42.076      |62.408      
SMOKE -> Precision: 0.4948 - Recall: 0.7606 - Accuracy: 0.5026 - F1: 0.5996
FIRE -> Precision: 0.3021 - Recall: 1.0000 - Accuracy: 0.3021 - F1: 0.4640


In [30]:
logger.info('\nTesting with FULL TEST LOADER')  
#logger.info(val_losses)
logger.info(val_metrics)

### Train DFire MINI

In [31]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=train_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 48.59it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
1.490       |0.569       |0.920       
SMOKE -> Precision: 0.7500 - Recall: 0.7600 - Accuracy: 0.6300 - F1: 0.7550
FIRE -> Precision: 0.3500 - Recall: 1.0000 - Accuracy: 0.3500 - F1: 0.5185


In [32]:
logger.info('\nTesting with DFire MINI TRAIN after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

### Test DFire MINI

In [33]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 44.65it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
1.491       |0.621       |0.870       
SMOKE -> Precision: 0.6364 - Recall: 0.6364 - Accuracy: 0.4667 - F1: 0.6364
FIRE -> Precision: 0.3667 - Recall: 1.0000 - Accuracy: 0.3667 - F1: 0.5366


In [34]:
logger.info('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

# Convert the Model to BIPOLAR OUT

In [35]:
import brevitas.nn as qnn
import torch.nn as nn

In [36]:
class CNV_BIPOLAR_OUT(nn.Module):
    def __init__(self, base_model):
        super(CNV_BIPOLAR_OUT, self).__init__()
        self.base_model = base_model
        self.qnt_output = qnn.QuantIdentity(
            quant_type='binary', 
            scaling_impl_type='const',
            bit_width=1, min_val=-1.0, max_val=1.0)

    def forward(self, x):
        x = self.base_model(x)
        x = self.qnt_output(x)
        return x

In [37]:
cnv_bipolar_out = CNV_BIPOLAR_OUT(model).to(config.DEVICE)

### New Evaluation for BIPOLAR Out Model

In [38]:
from tqdm import tqdm

def eval_bipolar_fn(loader, model, device):
    
    model.eval()
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        yhat = model(x)

        # print(y.shape)
        # print(yhat.shape)
        
        yhat[yhat < 1] = 0
    
        metrics.precision_metric.update(yhat, y)
        metrics.recall_metric.update(yhat, y)
        metrics.accuracy_metric.update(yhat, y)
        metrics.f1_metric.update(yhat, y)
   
    precision = metrics.precision_metric.compute()
    recall = metrics.recall_metric.compute()
    accuracy = metrics.accuracy_metric.compute()
    f1 = metrics.f1_metric.compute()
    
    metrics.precision_metric.reset()
    metrics.recall_metric.reset()
    metrics.accuracy_metric.reset()
    metrics.f1_metric.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()] 
        }
    )

In [39]:
logger.info("\n###############################################################")
logger.info("                 Results of BIPOLAR OUT Model")
logger.info("###############################################################")

### Full DS

In [40]:
cnv_bipolar_out.eval()
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=val_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.20it/s]

SMOKE -> Precision: 0.4948 - Recall: 0.7606 - Accuracy: 0.5026 - F1: 0.5996
FIRE -> Precision: 0.3021 - Recall: 1.0000 - Accuracy: 0.3021 - F1: 0.4640


In [41]:
logger.info('\nTesting with FULL TEST LOADER')  
logger.info(val_metrics)

### Mini Train

In [42]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=train_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 49.11it/s]

SMOKE -> Precision: 0.7500 - Recall: 0.7600 - Accuracy: 0.6300 - F1: 0.7550
FIRE -> Precision: 0.3500 - Recall: 1.0000 - Accuracy: 0.3500 - F1: 0.5185


In [43]:
logger.info('\nTesting with DFire MINI TRAIN after LOADING F1 Best Mean CHECKPOINT')  
logger.info(val_metrics)

### Mini Test

In [44]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=test_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 44.58it/s]

SMOKE -> Precision: 0.6364 - Recall: 0.6364 - Accuracy: 0.4667 - F1: 0.6364
FIRE -> Precision: 0.3667 - Recall: 1.0000 - Accuracy: 0.3667 - F1: 0.5366


In [45]:
logger.info('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
logger.info(val_metrics)

# Export Bipolar to QONNX

In [46]:
save_f1_name = 'best_mean_F1'
save_bipolar_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '__BIPOLAR_Out'
utils.export_onnx(cnv_bipolar_out, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_bipolar_onnx, config.DEVICE)

Model exported to ONNX: experiments_brevitas/test_v22_mini_resnet_70k_112_128_ds/onnx/Mobilenetv2_Mini_Resnet_4bitINP_classifier__best_mean_F1__BIPOLAR_Out
